In [3]:
import math

In [4]:
def F(B,C,D):
    return (B & C) | (~B & D)

def G(B,C,D):
    return (B & D) | (C & ~D)

def H(B,C,D):
    return B ^ C ^ D

def I(B,C,D):
    return C ^ (B | ~D)

In [5]:
def custom_md5(m: str, h0: int) -> str:
    # Argumentos:
    #    m:   str - mensaje
    #    h0: int - constante  inicial  H_0
    # Retorna:
    #    str - hash  MD5  correcto  del  mensaje  en  formato  hexadecimal
    
    # PADDING
    m = bytearray(m, encoding='utf-8')
    orig_len_in_bits = (8 * len(m)) & 0xffffffffffffffff
    m.append(0x80)
    while len(m) % 64 != 56:
        m.append(0)
    m += orig_len_in_bits.to_bytes(8, byteorder='little')
    chunks = [m[i:i+64] for i in range(0, len(m), 64)]
    
    # Define s and K
    s = [0 for i in range(64)] 
    s[ 0:16] = [ 7, 12, 17, 22,  7, 12, 17, 22,  7, 12, 17, 22,  7, 12, 17, 22 ]
    s[16:32] = [ 5,  9, 14, 20,  5,  9, 14, 20,  5,  9, 14, 20,  5,  9, 14, 20 ]
    s[32:48] = [ 4, 11, 16, 23,  4, 11, 16, 23,  4, 11, 16, 23,  4, 11, 16, 23 ]
    s[48:64] = [ 6, 10, 15, 21,  6, 10, 15, 21,  6, 10, 15, 21,  6, 10, 15, 21 ]
    
    K = [0 for i in range(64)]
    for i in range(64):
        K[i] = math.floor(2**32 * abs(math.sin(i + 1)))
    
    A = (h0 // 2**(32*3)) & 0xFFFFFFFF
    B = (h0 // 2**(32*2) - A * 2**32) & 0xFFFFFFFF
    C = (h0 // 2**(32) - A * 2**(32*2) - B * 2**32) & 0xFFFFFFFF
    D = (h0 - A * 2**(32*3) - B * 2**(32*2) - C * 2**32) & 0xFFFFFFFF
    
    for chunk in chunks:
        M = [chunk[chk:chk+4] for chk in range(0, 64, 4)]
        a,b,c,d = A,B,C,D
        # Main loop
        for i in range(64):
            f = ''
            g = 0
            if 0 <= i <= 15:
                f = F(b,c,d)
                g = i
            elif 16 <= i <= 31:
                f = G(b,c,d)
                g = (5*i + 1) % 16
            elif 32 <= i <= 47:
                f = H(b,c,d)
                g = (3*i + 5) % 16
            elif 48 <= i <= 63:
                f = I(b,c,d)
                g = (7*i) % 16
            f = (f + a + K[i] + int.from_bytes(M[g], byteorder='little')) & 0xFFFFFFFF
            a = d
            d = c
            c = b
            b = (b + (((f << s[i]) & 0xFFFFFFFF) | (f >> (32-s[i])))) & 0xFFFFFFFF
        A = (A + a) & 0xFFFFFFFF
        B = (B + b) & 0xFFFFFFFF
        C = (C + c) & 0xFFFFFFFF
        D = (D + d) & 0xFFFFFFFF
    final_value = A + (B << 32) + (C << 32*2) + (D << 32*3)
    digest = final_value.to_bytes(16, byteorder='little')
    return f'{int.from_bytes(digest, byteorder="big"):032x}'

In [6]:
custom_md5('hola', 12446846845685)

'd1f40d5352cc414e87d68560c22054b0'